# Лабораторная работа №3
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [5]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

def prefix_func(line):
    p = [0 for i in range(len(line))]
    j = 0
    i = 1
    while i < len(line):
        if line[i] != line[j]:
            if j == 0:
                p[i] = 0
                i += 1
            else:
                j = p[j - 1]
        else:
            p[i] = j + 1
            j += 1
            i += 1
    return p

def kmp_search(line, subline, reg):
    if len(line) < len(subline):
        return -1
    if reg == 0:
        line = line.lower()
        subline = subline.lower()
    p = prefix_func(subline)
    j = 0
    i = 0
    while i < len(line):
        if line[i] == subline[j]:
            i += 1
            j += 1
            if j == len(subline):
                return i - len(subline)
        else:
            if j > 0:
                j = p[j - 1]
            else:
                i += 1
    return -1
t1 = t.now()
print(kmp_search("борабора бораборо", "аборо", 0))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))
t1 = t.now()
print("борабора бораборо".find("аборо"))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))

12
--- 998 ms ---
12
--- 0 ms ---


#### Упрощенный алгоритм Бойера-Мура

In [6]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

def bm_search(line, subline, reg):
    if len(line) < len(subline):
        return -1
    if reg == 0:
        line = line.lower()
        subline = subline.lower()
    d = defaultdict()
    s = set()
    N = len(line)
    M = len(subline)
    for i in range(M - 2, -1, -1):
        if subline[i] not in s:
            d[subline[i]] = M - i - 1
            s.add(subline[i])
    if subline[M - 1] not in s:
        d[subline[M - 1]] = M
    d['*'] = M
    i = M - 1
    while i < N:
        k = 0
        for j in range(M - 1, -1, -1):
            if line[i - k] != subline[j]:
                if j == M - 1:
                    offset = d[line[i]] if d.get(line[i], False) else d['*']
                else:
                    offset = d[subline[j]]
                i += offset
                break
            k += 1
        
        if j == 0:
            return i - k + 1
    return -1


t1 = t.now()
print(bm_search("борабора бораборо", "аборо", 0))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))
t1 = t.now()
print("борабора бораборо".find("аборо"))
t2 = t.now()
delta = t2 - t1
print("--- {0} ms ---".format(delta.microseconds))

12
--- 0 ms ---
12
--- 0 ms ---


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [9]:
from heapq import heappush, heappop
import math
import copy
def solve(data):
    sum = 0
    zero_coord = 0
    #проверка на наличие решения
    for i in range(len(data)):
        if data[i] == 0:
            zero_coord = i
            continue
        for j in range(i + 1, len(data)):
            if data[i] > data[j] and data[j] != 0:
                sum += 1
    if len(data) % 3 != 0:
        sum += zero_coord // math.sqrt(len(data)) + 1
    if sum % 2 != 0:
        return []
    #есть наличие, решаем
    else:
        start = chain(data)
        temp = copy.deepcopy(data)
        temp.sort()
        del temp[0]
        temp.append(0)
        end = chain(temp)
        result = a_star(start, end.last_node())
        for node in result.history:
            print(node)
        print(result)
        return 1
    
def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

def manh_dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

class chain:
    # переопределяем str чтобы вывелась норм матрица
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size * self.size:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == self.size - 1:
                sstr += "\n"
            i += 1
        return sstr
    
    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(math.sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)
    # считаем манхэтеннское расстояние
    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)
    #проверяем можно ли за одни ход закончить или же прибавляем 2
    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2
    # эврестическое предположение о расстоянии от текущей вершины до конечной
    def h(self):
        return self.manh_dst() + self.last_move()
    # сумма расстояний от начальной вершины до текущей
    def g(self):
        return len(self.history)
    # вычисление эвристической функции
    def f(self):
        return self.h() # + self.g() , если хотим найти оптимальный маршрут, но потребуется гораздо больше времени
    
    def last_node(self):
        return str(self.board_state)
    # переопределение __It__ для сравнения двух цепей (сравниваются по значению эвристической функции)
    def __lt__(self, other):
        return self.f() < other.f()
    
    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # проверяем соседей и создаем новые ветви 
        if zero_coord + 1 < self.size * self.size and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord + self.size < self.size * self.size and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        return neighs
t1 = t.now()
print(solve([4, 12, 6, 2, 1, 9, 11, 15, 5, 13, 14, 7, 10, 3, 8, 0]))
#[0, 3, 5, 8, 6, 7, 1, 2, 4] <1s
#[15, 8, 10, 2, 3, 6, 5, 1, 7, 9, 11, 12, 0, 4, 13, 14] в отличие от A*,который вообще не может найти решение за 10+ минут
#даже при использовании свыше 2000 MB RAM, а  жадный нашёл решение за 28 секунд o_O
#[4, 12, 6, 2, 1, 9, 11, 15, 5, 13, 14, 7, 10, 3, 8, 0] жадный 1s31ms
#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 13, 14, 0] <1s
#[12, 5, 8, 7, 4, 11, 2, 14, 13, 6, 1, 0, 10, 9, 15, 3] 1m26s или 55s при жадном алгоритме
t2 = t.now()
delta = t2 - t1
print(delta)

4 12 6 2 
1 9 11 15 
5 13 14 7 
10 3 8 0 

4 12 6 2 
1 9 11 15 
5 13 14 7 
10 3 0 8 

4 12 6 2 
1 9 11 15 
5 13 0 7 
10 3 14 8 

4 12 6 2 
1 9 11 15 
5 13 7 0 
10 3 14 8 

4 12 6 2 
1 9 11 15 
5 13 7 8 
10 3 14 0 

4 12 6 2 
1 9 11 15 
5 13 7 8 
10 3 0 14 

4 12 6 2 
1 9 11 15 
5 13 7 8 
10 0 3 14 

4 12 6 2 
1 9 11 15 
5 0 7 8 
10 13 3 14 

4 12 6 2 
1 9 11 15 
5 7 0 8 
10 13 3 14 

4 12 6 2 
1 9 0 15 
5 7 11 8 
10 13 3 14 

4 12 6 2 
1 9 15 0 
5 7 11 8 
10 13 3 14 

4 12 6 2 
1 9 15 8 
5 7 11 0 
10 13 3 14 

4 12 6 2 
1 9 15 8 
5 7 0 11 
10 13 3 14 

4 12 6 2 
1 9 15 8 
5 7 3 11 
10 13 0 14 

4 12 6 2 
1 9 15 8 
5 7 3 11 
10 13 14 0 

4 12 6 2 
1 9 15 8 
5 7 3 0 
10 13 14 11 

4 12 6 2 
1 9 15 8 
5 7 0 3 
10 13 14 11 

4 12 6 2 
1 9 0 8 
5 7 15 3 
10 13 14 11 

4 12 0 2 
1 9 6 8 
5 7 15 3 
10 13 14 11 

4 12 2 0 
1 9 6 8 
5 7 15 3 
10 13 14 11 

4 12 2 8 
1 9 6 0 
5 7 15 3 
10 13 14 11 

4 12 2 8 
1 9 6 3 
5 7 15 0 
10 13 14 11 

4 12 2 8 
1 9 6 3 
5 7 15 11 
10 13 14 0 

4 12 2 8 
1

### Вывод

я написала лабу. урааа